## GeoCat experimental

In [ ]:
import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import timeit
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)

In [ ]:
df_checkin=pd.read_csv("../data/checkin/lasvegas.csv",nrows=9900,converters={'categories':string_to_array})
users=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
df_checkin.set_index("user_id",inplace=True)
df_checkin.head()

In [ ]:
df_poi=pd.read_csv("../data/poi/lasvegas.csv",converters={'categories':string_to_array})

In [ ]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [ ]:
users.count()

In [ ]:
N=80
K=20
VERY_SMALL_VALUE=-100
i=0

# tmp_rec_list=rec.mostpopular(df_checkin,K)
# tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
# print(tmp_rec_list)

for user_id in users:
    tmp_rec_list=rec.mostpopular(df_checkin.reset_index(),N)
    tmp_rec_list['score']=tmp_rec_list['score']/tmp_rec_list['score'].max()
    #print(tmp_rec_list)
    # add other columns
    tmp_rec_list=pd.merge(tmp_rec_list,df_poi,on='business_id').drop_duplicates(subset=df_poi.columns.difference(['categories'])).reset_index(drop=True)
    #print(tmp_rec_list)
    
    # User visits
    df_user_checkin=df_checkin.loc[[user_id]].reset_index(drop=True)

    business_cover=dict()
    for business_id in df_user_checkin['business_id'].drop_duplicates().reset_index(drop=True):
        business_cover[business_id]=0
    # Constants
    #COVER_OF_POI=len(df_user_review)/len(tmp_rec_list)
    div_geo_cat_weight = 0.5 # beta,this is here because of the work to be done on parameter customization for each user
    div_weight = 0.75 # lambda, geo vs cat
    current_proportionality=0
    
    rec_list=pd.DataFrame(columns=['business_id','score','latitude','longitude','categories'])
    relevant_categories_user=geocat.relevant_categories_to_the_user(df_user_checkin)
    
    
    
    # post-processing for diversification
    for i in range(K):
        start = timeit.default_timer()
        #num_current_candidates=len(tmp_rec_list)
        poi_to_insert=None
        max_objective_value=VERY_SMALL_VALUE
        rec_list_categories=geocat.category_in_rec_list(tmp_rec_list)
        
        for index,poi_candidate in tmp_rec_list.iterrows():
            objective_value=geocat.objective_ILD_GC_PR(poi_candidate,df_user_checkin,tmp_rec_list,K,\
                                                business_cover,current_proportionality,\
                                               div_geo_cat_weight,div_weight,dict_alias_title,undirected_category_tree,\
                                                      relevant_categories_user,rec_list_categories)
            
            if objective_value > max_objective_value:
                max_objective_value=objective_value
                poi_to_insert=poi_candidate
        #print(poi_to_insert)    
        stop = timeit.default_timer()
        print('Time:', stop - start)
        if poi_to_insert is not None:
            poi_to_insert['score']=max_objective_value
            rec_list=rec_list.append(poi_to_insert)
            # remove from tmp_rec_list
            tmp_rec_list=tmp_rec_list[tmp_rec_list['business_id']!=poi_to_insert['business_id']].reset_index(drop=True)
            
            current_proportionality=geocat.update_geo_cov(poi_to_insert,df_user_checkin,K,business_cover)
    
    print(user_id)        
    print(rec_list[['business_id','score']])
    break
    
    if i % 1111 ==10:
        break



0   IMLrj2klosTFvPRLv56cng  0.260358
24  ujmEBvifdJM6h6RLv4wQIg  0.050891
68  V13IWgZvNps2Vo7WyEnD8A  0.024049
0   XXW_OFaYQkkGOGniujZFHg  0.000000
0   FvVSy2r7_zDEhZWqLgjXNQ  0.000000
0   WRQ-9LluyivReFiQZFUujw  0.000000
0   Vs7gc9EE3k9wARuUcN9piA  0.000000
0   Os1n1_idfw9vv9kwULGJnQ  0.000000
0   qsJkjQoM3Hdg6Gtqp2S9_w  0.000000
0   O7UMzd3i-Zk8dMeyY9ZwoA  0.000000
0   qmymSqVwHYRqdwfcBatzpQ  0.000000
0   pH0BLkL4cbxKzu471VZnuA  0.000000
0   78hGCrQ6wLA2yh8zMywnmw  0.000000
0   _pBXtjN43eqMV0XZTz7nmw  0.000000
0   tMYYSWyTppDcF070Hmy-kQ  0.000000
0   dm6sO_Y8JdKTE1ZM955yug  0.000000
0   Yiw2jUJzUdFmglHZgj4RaQ  0.000000
0   FhIeCF6QrsLaRvAeu0oEPQ  0.000000
0   5ma1OUQWy_Ds80xUiP7JPQ  0.000000
0   k2b3niokS_tosjah_rzCPw  0.000000